In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [3]:

from dask_ml.model_selection import train_test_split
import dask.array as da
import numpy as np
from utils import dasker, config as util_config
from dask_ml.preprocessing import StandardScaler

In [5]:

client = dasker.get_dask_client()


# X and y must be Dask dataframes or arrays
num_obs = 1e5
num_features = 20
X = da.random.random(size=(num_obs, num_features), chunks=(1000, num_features))
y = da.random.random(size=(num_obs, 1), chunks=(1000, 1))




In [6]:
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True)

In [7]:
X_train.shape[1]

20

In [12]:
from keras import models, layers
model = models.Sequential()
model.add(layers.Dense(1, activation='linear', input_shape=(X_train.shape[1],)))

model.compile(optimizer='sgd',
              loss='mse',
              metrics=['mse'])

In [16]:
from keras.utils import Sequence

class DaskGenerator(Sequence):
    def __init__(self, samples, classes):
        '''Initialize a generator of samples and classes for training'''
        self.sample_batches = samples.to_delayed()
        self.class_batches = classes.to_delayed()
        
        assert len(self.sample_batches) == len(self.class_batches), 'lengths of samples and classes do not match'
        assert self.sample_batches.shape[1] == 1, 'all columns should be in each chunk'
    
    def __len__(self):
        '''Total number of batches, equivalent to Dask chunks in 0th dimension'''
        return len(self.sample_batches)
        
    def __getitem__(self, idx):
        '''Extract and compute a single batch returned as (X, y)'''
        return self.sample_batches[idx,0].compute(),  self.class_batches[idx].compute()

train_gen = DaskGenerator(X_train, y_train)
test_gen = DaskGenerator(X_test, y_test)

In [15]:
history = model.fit(X, y, 
                              #validation_data=test_gen,
                              
                              epochs=10,
                              
                              # use_multiprocessing=False,
                              # Not working for me; causes core dumps
                              #use_multiprocessing=True,
                              #workers=16,
                              #max_queue_size=5,
                              
                              verbose=1)

Epoch 1/10
3125/3125 [==============================] - 443s 141ms/step - loss: 0.1032 - mse: 0.1032
Epoch 2/10
3125/3125 [==============================] - 453s 145ms/step - loss: 0.0837 - mse: 0.0837
Epoch 3/10
3125/3125 [==============================] - 419s 134ms/step - loss: 0.0837 - mse: 0.0837
Epoch 4/10
3125/3125 [==============================] - 387s 124ms/step - loss: 0.0837 - mse: 0.0837
Epoch 5/10
3125/3125 [==============================] - 353s 113ms/step - loss: 0.0837 - mse: 0.0837
Epoch 6/10
2176/3125 [===================>..........] - ETA: 1:46 - loss: 0.0838 - mse: 0.0838

2022-09-10 13:30:22,464 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


2213/3125 [====================>.........] - ETA: 1:42 - loss: 0.0838 - mse: 0.0838

2022-09-10 13:30:26,875 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


2254/3125 [====================>.........] - ETA: 1:38 - loss: 0.0838 - mse: 0.0838

...

1983/3125 [==================>...........] - ETA: 2:08 - loss: 0.0837 - mse: 0.0837

2022-09-10 13:53:14,544 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


2029/3125 [==================>...........] - ETA: 2:03 - loss: 0.0838 - mse: 0.0838

2022-09-10 13:53:19,321 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


3125/3125 [==============================] - 351s 112ms/step - loss: 0.0838 - mse: 0.0838


In [16]:
score = model.evaluate(X, y, verbose=0)

[autoreload of scipy.sparse.linalg._isolve.iterative failed: Traceback (most recent call last):
  File "/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/vagrant/miniconda3/envs/test/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/vagrant/miniconda3/envs/test/lib/python3.8/site-packages/scipy/sparse/linalg/_isolve/iterative.py", line 150, in <module>
    def bicg(A, b, x0=None, tol=1e-5, maxiter=None, M=None, callback=None, atol=None):
  Fi